In [1]:
%pip install ninja cmake
%pip install torch torchvision torchaudio
%pip install --no-build-isolation mamba-ssm

   ---------------------------------------- 0.0/37.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/37.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/37.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/37.6 MB ? eta -:--:--
    --------------------------------------- 0.5/37.6 MB 837.5 kB/s eta 0:00:45
    --------------------------------------- 0.8/37.6 MB 931.2 kB/s eta 0:00:40
   - -------------------------------------- 1.0/37.6 MB 914.5 kB/s eta 0:00:40
   - -------------------------------------- 1.3/37.6 MB 944.7 kB/s eta 0:00:39
   - -------------------------------------- 1.3/37.6 MB 944.7 kB/s eta 0:00:39
   - -------------------------------------- 1.6/37.6 MB 864.6 kB/s eta 0:00:42
   - -------------------------------------- 1.8/37.6 MB 915.0 kB/s eta 0:00:40
   - -------------------------------------- 1.8/37.6 MB 915.0 kB/s eta 0:00:40
   -- ------------------------------------- 2.1/37.6 MB 939.6 kB/s eta 0:00:38
   -- ---

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      
      
      torch.__version__  = 2.8.0+cu128
      
      
      <string>:118: UserWarning: mamba_ssm was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
      Traceback (most recent call last):
        File "c:\Users\USER\Desktop\RA\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
        File "c:\Users\USER\Desktop\RA\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\USER\Desktop\RA

In [ ]:
CONFIG = {
     "MODEL_PATHS": {
         "conformer": "C:\Users\USER\Desktop\RA\ensemble\best_conformer_default.ptt",
         "stmambanet": "C:\Users\USER\Desktop\RA\ensemble\best_model_mamba.pt",
         "conformer_opt": "C:\Users\USER\Desktop\RA\ensemble\best_conformer_with_freq.pt",
         "eegnet": None  # Path to saved EEGNet model (TensorFlow)
     },
     "DATA_PATH_CH": "C:\Users\USER\Desktop\RA\data_ch",
     "DATA_PATH": "C:\Users\USER\Desktop\RA\data"
}

In [22]:
import sys
# Add the directory containing custom modules to the Python path
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/')

# Verify that the path was added
print(f"Updated sys.path: {sys.path}")

Updated sys.path: ['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/', '/content/dri

In [23]:
import torch
import numpy as np
from torch import nn
from collections import Counter
import torch.nn.functional as F
import os
from scipy import signal

# PyTorch specific imports for data handling and training
from torch.utils.data import DataLoader, TensorDataset, Dataset
from tqdm.notebook import tqdm

# Import model architectures
from conformer import Conformer
from mamba_ssm import Mamba
from conformer_opt import ConformerWithFreq

# Data processing imports
import h5py
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

ModuleNotFoundError: No module named 'conformer'

In [18]:
class ConvEncoder(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1, padding=0):
        super().__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.norm = nn.BatchNorm1d(out_channels)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = x.transpose(1, 2)  # [B, T, C] → [B, C, T]
        x = self.conv(x)
        x = self.norm(x)
        x = self.activation(x)
        x = x.transpose(1, 2)  # [B, C, T] → [B, T, C]
        return x

class STMambaNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(STMambaNet, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.kernel_sizes = [5, 9]
        self.conv_channels = [4, 4]

        self.temp_convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=out_ch, kernel_size=(1, k), padding=(0, k // 2))
            for k, out_ch in zip(self.kernel_sizes, self.conv_channels)
        ])

        self.total_conv_channels = sum(self.conv_channels)
        self.batch_norm1 = nn.BatchNorm2d(self.total_conv_channels)
        self.spatial_conv = nn.Conv2d(in_channels=self.total_conv_channels, out_channels=self.total_conv_channels, kernel_size=(1, 1))
        self.batch_norm2 = nn.BatchNorm2d(self.total_conv_channels)
        self.activation = nn.ELU()

        self.var_pool = lambda x: torch.var(x, dim=1, keepdim=True)
        self.avg_pool = lambda x: torch.mean(x, dim=1, keepdim=True)

        # Размерность нормализации соответствует сохраненным весам
        self.norm_t = nn.LayerNorm(55)  # В сохраненных весах это 55
        self.norm_s = nn.LayerNorm(100) # В сохраненных весах это 100
        self.dropout = nn.Dropout(0.3)

        self.feedforward_1 = nn.Sequential(
            nn.Linear(55, 32),
            nn.ReLU(),
            nn.Linear(32, 55)
        )
        self.feedforward_2 = nn.Sequential(
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 100)
        )

        self.mamba_t = Mamba(d_model=55, d_state=8)
        self.mamba_s = Mamba(d_model=100, d_state=8)

        self.conv_t = ConvEncoder(in_channels=110, out_channels=55)
        self.conv_s = ConvEncoder(in_channels=200, out_channels=100)

        self.pool_t = nn.Identity()
        self.pool_s = nn.Identity()

        self.fc_s = nn.Linear(55*100, 32)
        self.fc_t = nn.Linear(100 * 55, 32)

        self.fc_class = nn.Linear(64, num_classes)

    def forward(self, x, mode="classification"):
        x = x.unsqueeze(1)
        x_convs = [conv(x) for conv in self.temp_convs]
        x = torch.cat(x_convs, dim=1)
        x = self.batch_norm1(x)
        x = self.spatial_conv(x)
        x = self.batch_norm2(x)
        x = self.activation(x)
        x_var = self.var_pool(x)
        x_avg = self.avg_pool(x)
        x_var = x_var.permute(0, 2, 3, 1).squeeze(-1)
        x_avg = x_avg.permute(0, 2, 3, 1).squeeze(-1)

        def shared_mamba_block(x, mamba, norm, feedforward):
            res1 = x
            x = norm(x)
            x = mamba(x)
            x = self.dropout(x) + res1
            res2 = x
            x = norm(x)
            x = feedforward(x) + res2
            return x

        x_tvar = shared_mamba_block(x_var.permute(0, 2, 1), self.mamba_t, self.norm_t, self.feedforward_1)
        x_tavg = shared_mamba_block(x_avg.permute(0, 2, 1), self.mamba_t, self.norm_t, self.feedforward_1)
        x_t = torch.cat([x_tvar, x_tavg], dim=-1)
        x_svar = shared_mamba_block(x_var, self.mamba_s, self.norm_s, self.feedforward_2)
        x_savg = shared_mamba_block(x_avg, self.mamba_s, self.norm_s, self.feedforward_2)
        x_s = torch.cat([x_svar, x_savg], dim=-1)
        x_tconv = self.conv_t(x_t)
        x_sconv = self.conv_s(x_s)
        x_sconv = x_sconv.permute(0, 2, 1)
        x_tconv = self.pool_t(x_tconv).reshape(x_tconv.shape[0], -1)
        x_sconv = self.pool_s(x_sconv).reshape(x_sconv.shape[0], -1)
        x_sfc = self.fc_s(x_sconv)
        x_tfc = self.fc_t(x_tconv)
        x_fused = torch.cat([x_sfc, x_tfc], dim=1)

        if mode == "classification":
            return self.fc_class(x_fused)

# --- FIX: Simplified EEGNetv2 without subject ID handling ---
class EEGNetv2(nn.Module):
    """
    PyTorch implementation of EEGNet. This version is generalized and does not use subject IDs.
    """
    def __init__(self, nb_classes=2, Chans=55, Samples=100, dropoutRate=0.5,
                 F1=16, D=2, F2=16, kernel_length=10):
        super(EEGNetv2, self).__init__()
        self.Chans = Chans
        self.Samples = Samples

        # Block 1: Temporal and Depthwise Convolutions
        self.conv1 = nn.Conv2d(1, F1, (1, kernel_length), padding=(0, kernel_length // 2), bias=False)
        self.bn1 = nn.BatchNorm2d(F1)
        self.depthwise_conv = nn.Conv2d(F1, F1 * D, (Chans, 1), groups=F1, bias=False)
        self.bn2 = nn.BatchNorm2d(F1 * D)
        self.activation_elu = nn.ELU()
        self.avg_pool1 = nn.AvgPool2d((1, 4))
        self.dropout1 = nn.Dropout(dropoutRate)

        # Block 2: Separable Convolution
        self.separable_conv = nn.Sequential(
            nn.Conv2d(F1 * D, F1 * D, (1, kernel_length), groups=F1 * D, padding=(0, kernel_length // 2), bias=False),
            nn.Conv2d(F1 * D, F2, 1, bias=False)
        )
        self.bn3 = nn.BatchNorm2d(F2)
        self.avg_pool2 = nn.AvgPool2d((1, 4))
        self.dropout2 = nn.Dropout(dropoutRate)

        # Classifier
        flattened_size = F2 * (Samples // 4 // 4)
        self.fc = nn.Linear(flattened_size, nb_classes)

    def apply_constraints(self):
        w = self.depthwise_conv.weight.data
        norm = w.norm(2, dim=(0, 2, 3), keepdim=True)
        desired = torch.clamp(norm, 0, 1.0)
        w.copy_((w * desired / (1e-8 + norm)))

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.depthwise_conv(x)
        x = self.bn2(x)
        x = self.activation_elu(x)
        x = self.avg_pool1(x)
        x = self.dropout1(x)
        x = self.separable_conv(x)
        x = self.bn3(x)
        x = self.activation_elu(x)
        x = self.avg_pool2(x)
        x = self.dropout2(x)
        x_flat = torch.flatten(x, 1)
        output = self.fc(x_flat)
        return output

In [ ]:
class EnsembleModel:
    """
    Ансамблевая модель для классификации памяти с использованием мажоритарного голосования.
    Все модели реализованы на PyTorch для избежания конфликтов фреймворков.
    """
    def __init__(self, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.device = device
        self.models = {}
        self.model_paths = CONFIG["MODEL_PATHS"]

        # Параметры канальной выборки для разных датасетов
        self.ch_ch = list(range(63))
        self.g_ch = list(range(61))
        self.ch_rem = [0, 2, 6, 8, 29, 31, 51, 55]
        self.g_rem = [46, 49, 56, 58, 59, 60]

        for i in self.ch_rem:
            self.ch_ch.remove(i)
        for j in self.g_rem:
            self.g_ch.remove(j)

    def load_models(self):
        """Загрузка моделей из сохраненных весов."""
        # Загрузка EEG-Conformer
        try:
            self.models["conformer"] = Conformer(emb_size=40, depth=6, n_classes=2)
            state_dict = torch.load(self.model_paths["conformer"], map_location=self.device, weights_only=True)
            self.models["conformer"].load_state_dict(state_dict, strict=False)
            self.models["conformer"].to(self.device)
            self.models["conformer"].eval()
            print("EEG-Conformer загружен успешно!")
        except Exception as e:
            print(f"Ошибка при загрузке EEG-Conformer: {e}")
            self.models["conformer"] = None

        # Загрузка STMambaNet
        try:
            self.models["stmambanet"] = STMambaNet(input_size=100, hidden_size=128, num_classes=2)
            self.models["stmambanet"].load_state_dict(torch.load(self.model_paths["stmambanet"], map_location=self.device, weights_only=True))
            self.models["stmambanet"].to(self.device)
            self.models["stmambanet"].eval()
            print("STMambaNet загружен успешно!")
        except Exception as e:
            print(f"Ошибка при загрузке STMambaNet: {e}")
            self.models["stmambanet"] = None

        # --- FIX: Load the simplified EEGNetv2 without subject ID parameters ---
        try:
            self.models["eegnet"] = EEGNetv2(nb_classes=2, Chans=55, Samples=100)
            if self.model_paths["eegnet"]:
                self.models["eegnet"].load_state_dict(torch.load(self.model_paths["eegnet"], map_location=self.device))
                print("EEGNet (PyTorch) загружен из файла успешно!")
            else:
                print("Создана новая модель EEGNet (PyTorch, не обучена)")
            self.models["eegnet"].to(self.device)
            self.models["eegnet"].eval()
        except Exception as e:
            print(f"Ошибка при загрузке/создании EEGNet: {e}")
            self.models["eegnet"] = None

    # --- FIX: Updated data loading to perform global undersampling and remove subject IDs ---
    def load_and_preprocess_data(self, data_path_ch, data_path, perform_balancing=False):
        """Загрузка и предобработка данных. Балансировка выполняется на всем наборе данных."""
        X_all, y_all = [], []
        for subj in range(1, 10):
            try:
                mat_data = loadmat(f"{data_path_ch}/ASK/sbj_{subj}.mat")
                X = np.array(mat_data['x_post'])[:-1, self.ch_ch, :100]
                y = np.array(mat_data['label_next_ind'])[0, :X.shape[0]]  # Обрезаем метки до размера данных
                print(f"Загружен субъект {subj} (CH): X={X.shape}, y={y.shape}")
                X_all.append(X); y_all.append(y)
            except Exception as e: print(f"Ошибка при загрузке файла sbj_{subj}.mat (CH): {e}")
        for subj in range(1, 15):
            try:
                with h5py.File(f"{data_path}/ASK/sbj_{subj}.mat", 'r') as f:
                    X = np.array(f['x_post'])[:, self.g_ch, :100]
                    y = np.array(f['label_next_ind'])[:X.shape[0], 0]  # Обрезаем метки до размера данных
                    print(f"Загружен субъект {subj} (KR): X={X.shape}, y={y.shape}")
                    X_all.append(X); y_all.append(y)
            except Exception as e: print(f"Ошибка при загрузке файла sbj_{subj}.mat (KR): {e}")

        data = np.concatenate(X_all, axis=0)
        labels = np.concatenate(y_all, axis=0)

        # Проверяем совпадение размеров X и y
        if data.shape[0] != labels.shape[0]:
            print(f"ВНИМАНИЕ: Несоответствие размеров данных ({data.shape[0]}) и меток ({labels.shape[0]})")
            min_samples = min(data.shape[0], labels.shape[0])
            data = data[:min_samples]
            labels = labels[:min_samples]
            print(f"Обрезано до {min_samples} образцов")

        data_flattened = data.reshape(data.shape[0], -1)
        scaler = StandardScaler()
        data_normalized = scaler.fit_transform(data_flattened).reshape(data.shape)

        if perform_balancing:
            print(f"Размер до балансировки: {Counter(labels)}")
            rus = RandomUnderSampler(random_state=42)
            data_shape = data_normalized.shape
            X_reshaped = data_normalized.reshape(data_shape[0], -1)
            X_res, y_res = rus.fit_resample(X_reshaped, labels)
            data_normalized = X_res.reshape(X_res.shape[0], *data_shape[1:])
            labels = y_res
            print(f"Размер после балансировки: {Counter(labels)}")

        return data_normalized, labels

    def _predict_torch_model(self, model_name, X):
        """Внутренний метод для предсказания любой PyTorch модели."""
        model = self.models.get(model_name)
        if model is None: return None

        model.eval()
        X_tensor = torch.from_numpy(X).float().to(self.device)

        with torch.no_grad():
            if model_name == "conformer":
                X_tensor = X_tensor.unsqueeze(1)
                _, outputs = model(X_tensor)
            elif model_name == "stmambanet":
                outputs = model(X_tensor, mode="classification")
            else: # EEGNet
                outputs = model(X_tensor)

            preds = torch.argmax(F.softmax(outputs, dim=1), dim=1).cpu().numpy()
        return preds

    def predict_conformer(self, X): return self._predict_torch_model("conformer", X)
    def predict_stmambanet(self, X): return self._predict_torch_model("stmambanet", X)
    def predict_eegnet(self, X): return self._predict_torch_model("eegnet", X)

    # --- FIX: Simplified training loop without subject IDs ---
    def train_eegnet(self, X_train, y_train, X_val, y_val,
                     epochs=50, batch_size=32, save_path=None, learning_rate=0.001):
        """Обучение EEGNet (PyTorch) модели."""
        model = self.models.get("eegnet")
        if model is None: return None

        model.to(self.device)
        train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataset = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

        for epoch in range(epochs):
            model.train()
            running_loss, correct_train, total_train = 0.0, 0, 0

            pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
            for data, labels in pbar:
                data, labels = data.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = model(data)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                if hasattr(model, 'apply_constraints'): model.apply_constraints()

                running_loss += loss.item() * data.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                pbar.set_postfix({'loss': f'{running_loss/total_train:.4f}', 'acc': f'{correct_train/total_train:.4f}'})

            history['loss'].append(running_loss / total_train)
            history['accuracy'].append(correct_train / total_train)

            model.eval()
            val_loss, correct_val, total_val = 0.0, 0, 0
            with torch.no_grad():
                for data, labels in val_loader:
                    data, labels = data.to(self.device), labels.to(self.device)
                    outputs = model(data)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * data.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).sum().item()

            history['val_loss'].append(val_loss / total_val)
            history['val_accuracy'].append(correct_val / total_val)
            print(f"Epoch {epoch+1} Summary: Val Loss: {val_loss/total_val:.4f}, Val Acc: {correct_val/total_val:.4f}")

        if save_path:
            torch.save(model.state_dict(), save_path)
            self.model_paths["eegnet"] = save_path
            print(f"Модель EEGNet сохранена по пути: {save_path}")
        return history

    def predict(self, X):
        """Мажоритарное голосование всех доступных моделей."""
        predictions = []
        model_names = []

        conformer_preds = self.predict_conformer(X)
        if conformer_preds is not None: predictions.append(conformer_preds); model_names.append("conformer")

        stmambanet_preds = self.predict_stmambanet(X)
        if stmambanet_preds is not None: predictions.append(stmambanet_preds); model_names.append("stmambanet")

        eegnet_preds = self.predict_eegnet(X)
        if eegnet_preds is not None: predictions.append(eegnet_preds); model_names.append("eegnet")

        if not predictions: raise ValueError("Нет доступных моделей для прогнозирования")

        stacked_preds = np.stack(predictions, axis=1)
        ensemble_preds = []
        for i in range(stacked_preds.shape[0]):
            votes = stacked_preds[i]
            vote_counts = Counter(votes)
            most_common = vote_counts.most_common()

            if len(most_common) > 1 and most_common[0][1] == most_common[1][1]:
                if "conformer" in model_names: ensemble_preds.append(votes[model_names.index("conformer")]); continue
                if "stmambanet" in model_names: ensemble_preds.append(votes[model_names.index("stmambanet")]); continue
                ensemble_preds.append(most_common[0][0])
            else:
                ensemble_preds.append(most_common[0][0])

        return np.array(ensemble_preds)

# --- FIX: Main execution block updated for simplified data handling ---
if __name__ == "__main__":
    ensemble = EnsembleModel()
    ensemble.load_models()

    try:
        print("\nЗагрузка и предобработка данных...")
        # Load the full, unbalanced dataset first
        X_full, y_full = ensemble.load_and_preprocess_data(CONFIG["DATA_PATH_CH"], CONFIG["DATA_PATH"], perform_balancing=False)
        print(f"Полный набор данных загружен. Форма X: {X_full.shape}, Форма y: {y_full.shape}")

        # Проверяем соответствие размеров перед разбиением
        if X_full.shape[0] != y_full.shape[0]:
            raise ValueError(f"Несоответствие размеров между X_full ({X_full.shape[0]}) и y_full ({y_full.shape[0]})")

        # Split the full dataset into training and testing sets
        # Stratify by labels to ensure test set is representative of the original data distribution
        X_train_unbalanced, X_test, y_train_unbalanced, y_test = train_test_split(
            X_full, y_full, test_size=0.2, random_state=42, stratify=y_full)

        print(f"Размер тестового набора: {X_test.shape}, Распределение: {Counter(y_test)}")

        # Now, balance ONLY the training data
        print("\nБалансировка только обучающего набора...")
        rus = RandomUnderSampler(random_state=42)
        X_train_reshaped = X_train_unbalanced.reshape(X_train_unbalanced.shape[0], -1)
        X_train_balanced_reshaped, y_train_balanced = rus.fit_resample(X_train_reshaped, y_train_unbalanced)
        X_train = X_train_balanced_reshaped.reshape(X_train_balanced_reshaped.shape[0], *X_train_unbalanced.shape[1:])
        y_train = y_train_balanced

        print(f"Размер обучающего набора после балансировки: {X_train.shape}, Распределение: {Counter(y_train)}")

        if ensemble.model_paths["eegnet"] is None:
            print("\nОбучение модели EEGNet...")
            X_train_split, X_val, y_train_split, y_val = train_test_split(
                X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

            history = ensemble.train_eegnet(
                X_train_split, y_train_split,
                X_val, y_val,
                epochs=10, batch_size=64,
                save_path='/content/drive/MyDrive/Colab Notebooks/Ensemble_Adlet/trained_eegnet_generalized.pt'
            )

            try:
                import matplotlib.pyplot as plt
                plt.figure(figsize=(12, 4))
                plt.subplot(1, 2, 1); plt.plot(history['loss'], label='train'); plt.plot(history['val_loss'], label='validation'); plt.title('Функция потерь'); plt.legend()
                plt.subplot(1, 2, 2); plt.plot(history['accuracy'], label='train'); plt.plot(history['val_accuracy'], label='validation'); plt.title('Точность'); plt.legend()
                plt.tight_layout()
                plt.savefig('/content/drive/MyDrive/Colab Notebooks/Ensemble_Adlet/eegnet_training_history_generalized.png')
                print("График истории обучения сохранен")
            except Exception as e: print(f"Не удалось создать график истории обучения: {e}")

        print("\nОценка каждой модели отдельно на тестовом наборе:")
        for name in ["conformer", "stmambanet", "eegnet"]:
            if ensemble.models.get(name):
                preds = ensemble._predict_torch_model(name, X_test)
                acc = np.mean(preds == y_test)
                print(f"Точность {name.capitalize()}: {acc:.4f}")

        print("\nОценка ансамблевой модели:")
        predictions = ensemble.predict(X_test)
        accuracy = np.mean(predictions == y_test)
        print(f"Точность ансамбля: {accuracy:.4f}")

        from sklearn.metrics import confusion_matrix, classification_report
        print("\nМатрица ошибок:")
        print(confusion_matrix(y_test, predictions))
        print("\nОтчет о классификации:")
        print(classification_report(y_test, predictions, target_names=["Класс 0", "Класс 1"]))

    except Exception as e:
        import traceback
        print(f"Произошла ошибка: {e}")
        traceback.print_exc()
        print("\nИспользование случайных данных для демонстрации...")
        X_demo = np.random.randn(10, 55, 100).astype(np.float32)
        predictions = ensemble.predict(X_demo)
        print("Ансамблевые предсказания (случайные данные):", predictions)

In [ ]:
!pip install --force-reinstall --no-build-isolation mamba-ssm

In [16]:
import os
print(os.listdir('/content/drive/MyDrive/Colab_Notebooks/Ensemble_Adlet/'))

['Сodes.zip', 'best_conformer_with_freq.pt', 'compare_classes.txt', 'ensemble_model.py', '.DS_Store', 'EEG-conformer.ipynb', 'conformer.py', 'BalancedAcrossSubjectsAllData.ipynb', 'data', 'data_ch', '__pycache__', 'STmambaNet_the_last_version_2.ipynb', 'best_model_mamba.pt', 'best_conformer_default.pt', 'Preprocessing.png', 'trained_eegnet_generalized.pt', 'eegnet_training_history_generalized.png', 'conformer_opt.py', 'Preprocess_freq.py', 'STmambaNet_the_last_version.ipynb', 'Untitled0.ipynb', 'Adlet_ensemble.ipynb']
